In [ ]:
import numpy as np
import sys
from sklearn.metrics import accuracy_score

In [ ]:
def read_csv(fn):
  with open(fn) as fd:
    data = np.array([
          [
            float(i) for i in line.split(',')
          ] 
          for line in fd.read().split('\n') if line!='']
        )
  return data

In [ ]:
def read_data(tn, tl):
    train_data = read_csv(tn)
    train_labels = read_csv(tl)
    return train_data, train_labels


In [ ]:
class CrossEntropy():
    def loss(self, y, y_hat):
        n = len(y)
        lp = - np.log(y_hat[np.arange(n), y.argmax(axis=1)])
        return np.sum(lp)/n
        
    def grad(self, y, y_hat):
        n = y.shape[0]
        res = y_hat - y
        return res/n

class Sigmoid():
  def __init__(self):
    self.name = 'sigmoid'
    
  def act(self, inp):  
    return 1 / (1 + np.exp(-inp))
  
  def grad(self, inp):
    return inp * (1 - inp)


In [ ]:
class SoftmaxActivation():
    def __init__(self):
        self.name = 'softmax'
        
    def act(self, inp):
        e = np.exp(inp - np.max(inp, axis=1, keepdims=True))
        return e / np.sum(e, axis=1, keepdims=True)

    def prime(self, inp):
        return inp * (1 - inp)
        
activations = {
    'sigmoid': Sigmoid,
    'softmax': SoftmaxActivation
}


In [ ]:
class FullyConnectedLayer():
  def __init__(self, input_dims, op_dims, activation):
    self.w = np.random.rand(input_dims, op_dims) 
    self.b = np.zeros((1, op_dims))
    self.act = activations[activation]()
    

  def fp(self, inp):
    self.inp = inp
    self.op = np.dot(self.inp, self.w) + self.b
    self.op = self.act.act(self.op)
    return self.op

  def bp(self, oe, lr):
    if self.act.name != 'softmax':
        oe = self.act.grad(self.op) * oe
    
    ie = np.dot(oe, self.w.T)
    we = np.dot(self.inp.T, oe)
    self.w -= lr * we

    if self.act.name == 'softmax':
        self.b -= lr * np.sum(oe, axis=0, keepdims=True)
    else:
        self.b -= lr * np.sum(oe, axis=0)
    return ie


In [ ]:
class Model:
  def __init__(self):
    self._los_ = None
    self.ls = []

  def add(self, layer):
    self.ls.append(layer)

  def loss(self, l):
    self._los_ = l

  def predict(self, inp):
    op = inp
    for layer in self.ls:
        op = layer.fp(op)
    return op
  
  def train(self, x, y, epcs, 
                          lr, bs=200):
    N = len(x)
    history = []
    for ep in range(epcs):
      err = 0
      for j in range(int(N/bs)):
        op = self.predict(
              x[j:j+bs]
            )
        err += self._los_.loss(y[j:j+bs], op)

        __err__ = self._los_.grad(y[j:j+bs], op)
        for l in reversed(self.ls):
            __err__ = l.bp(__err__, lr)

      if ep%100 == 0:
        print("Epoch {}/{}  error={}".format(ep+1, epcs, err))
      if err < 0.03:
        break
      history.append({
          'epoch': ep,
          'err': err
      })
    return history


In [ ]:
# Change file names here to test on different files.
tdn = 'gaussian_train_data.csv'
tln = 'gaussian_train_label.csv'
test_data_name = 'gaussian_test_data.csv'
test_label_name = 'gaussian_test_label.csv'
train_x, train_y = read_data(tdn, tln)
test_x, test_y = read_data(test_data_name, test_label_name)


a = train_y.astype(int).flatten()
train_y = np.zeros((a.size, a.max() + 1))
train_y[np.arange(a.size), a] = 1

net = Model()

net.add(FullyConnectedLayer(2, 128, 'sigmoid'))
net.add(FullyConnectedLayer(128, 128, 'sigmoid'))
net.add(FullyConnectedLayer(128, 2, 'softmax'))
net.loss(CrossEntropy())

hist = net.train(train_x, train_y, epcs=8000, lr=0.7)

preds = net.predict(test_x).argmax(axis=1)
preds_train = net.predict(train_x).argmax(axis=1)
print ("Train accuracy: ", accuracy_score(preds_train, a))
print ("Test accuracy: ", accuracy_score(preds, test_y))